In [1]:
import pandas as pd
import time
import random
import json
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import openpyxl
from selenium.webdriver.chrome.options import Options

from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
s=Service(ChromeDriverManager().install())
chrome_options = webdriver.ChromeOptions()
chrome_options.headless = False

browser = webdriver.Chrome(service=s, options = chrome_options)
wait = WebDriverWait(browser, 30)
browser.set_page_load_timeout(10)
#browser.maximize_window()

In [3]:
url_head = "https://www.estimize.com/users/sign_in"
browser.get(url_head)
#time.sleep(random.uniform(1,5))

In [4]:
def sign_in(url_head):
    browser.get(url_head)

    email_field = browser.find_element(By.ID, 'user_login')
    password_field = browser.find_element(By.ID, 'user_password')
    submit_btn = browser.find_element(By.XPATH, '//input[@type="submit"]')

    with open('estimizeCredentials.json') as f:
        d = json.loads(f.read())
        username = d['estimize'][0]['username']
        password = d['estimize'][0]['password']

    email_field.send_keys(username)
    password_field.send_keys(password)
    submit_btn.click()
    #time.sleep(1)

In [5]:
sign_in(url_head)

In [6]:
def getCompanyInformation(tickerListDF):
    for ind, row in tickerListDF.iterrows():
        stock = row["Ticker"]
        #stock = "PLAY"
        stock_URL = ("https://www.estimize.com/"+stock.lower()+"/fq4-2021?metric_name=eps&chart=historical")
        browser.get(stock_URL)
       
        ticker = browser.find_element(By.LINK_TEXT, stock).text
        name = browser.find_element(By.CSS_SELECTOR, 'p.release-header-information-description').text
        sectorInd = browser.find_elements(By.XPATH, '//span[@itemprop="title"]')
        sector = sectorInd[0].text
        industry = sectorInd[1].text
        numFollower = browser.find_element(By.XPATH, '//div[@class="estimize-row summary-sub-title"]').text
        numAnalyst = browser.find_element(By.XPATH, '//a[@class="estimize-row summary-sub-title"]').text
        
        Ticker_BI.append(ticker)
        Ticker_Name.append(name)
        Ticker_Sectors.append(sector)
        Ticker_Industries.append(industry)
        Ticker_numFollowers.append(int(numFollower.replace(',', '')))
        Ticker_NumAnalysts.append(int(numAnalyst.replace(',', '')))
        
        getEPSInformation(ticker)    

In [7]:
def getEPSInformation(ticker):
    browser.execute_script("window.scrollBy(0, 300)")
    wait.until(EC.element_to_be_clickable((By.XPATH, '//ul[@class="metrics-chart-menu"]//li[2]'))).click()
    
    [Ticker_EPS.append(ticker) for i in range(4)]

    quarterList = browser.find_elements(By.XPATH, '//a[@class="rel-chart-tbl-release-link"]')
    [Quarter_EPS.append(qtr.text) for qtr in quarterList[4:8]]

    rptEarnings = browser.find_elements(By.XPATH, '//tbody[@class="rel-chart-tbl-group rel-chart-tbl-group-actual"]//tr[1]//td')
    [ReportedEarnings_EPS.append(rpt.text) for rpt in rptEarnings[4:8]]

    estConsen = browser.find_elements(By.XPATH, '//tr[@class="rel-chart-tbl-estimize"][1]//td')
    [EstConsensus_EPS.append(est.text) for est in estConsen[4:8]]

    estMean = browser.find_elements(By.XPATH, '//tr[@class="rel-chart-tbl-estimize"][2]//td')
    [EstMean_EPS.append(estm.text) for estm in estMean[4:8]]

    wsConsen = browser.find_elements(By.XPATH, '//tbody[@class="rel-chart-tbl-group rel-chart-tbl-group-wall-street"]//tr[1]//td')
    [WSConsensus_EPS.append(wsCon.text) for wsCon in wsConsen[4:8]]
    
    getEPSAnalystsEstimation(ticker, quarterList[4:8])
    

In [8]:
def getEPSAnalystsEstimation(ticker, quarterList):
    quarters = []
    for i in quarterList:
        quarters.append(i.text)
        
    for qtr in quarters:
        time.sleep(0.5)
        eps_URL = ("https://www.estimize.com/"+ticker.lower()+"/"+(qtr[0:3]+"-20"+qtr[5:7]).lower()+"?metric_name=eps&chart=historical")
        browser.get(eps_URL)
        
        analystDisplayNameList = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//td[@class="estimates-tbl-analyst"]//a[@class="username"]')))
        for aun in analystDisplayNameList:
            Ticker_AE.append(ticker)
            Quarter_AE.append(qtr)
            AnalystDisplayName_AE.append(aun.text)
    
        analystSecondaryNameList = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//td[@class="estimates-tbl-analyst"]//a[@class="secondary"]')))
        for asn in analystSecondaryNameList:
            AnalystSecondaryName_AE.append(asn.get_attribute('href')[31:])
        
        analystEPSestimationList = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//td[@class="estimates-tbl-eps"]')))
        for aee in analystEPSestimationList:
            AnalystEPSestimation_AE.append(aee.text)

In [9]:
def get_Analyst_Information():
    analystUserName = wait.until(EC.presence_of_element_located((By.XPATH, '//p[@class="profile-username"]//a'))).get_attribute('href')[31:]
    AnalystUserName.append(analystUserName)
    
    analystRoleList = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//ul[@class="profile-bio-categorizations"]')))
    alystRole = ""
    if len(analystRoleList[0].text) > 0:
        for i in analystRoleList:
            alystRole = (alystRole +"_"+ i.text)
        AnalystRole.append(alystRole[1:])
    else:
        AnalystRole.append("No Job Desc")

    joinDateString = wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="profile-activity-stats"]')))
    matches = re.findall('((\d{2}|January|Jan|February|Feb|March|Mar|April|Apr|May|May|June|Jun|July|Jul|August|Aug|September|Sep|October|Oct|November|Nov|December|Dec)[\/ ]\d{2,4})', joinDateString.text)
    for match in matches:
        alystJoinDate = match[0]
    AnalystJoinDate.append(alystJoinDate)

    alystConfidenceRate = wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="value"]'))).text
    AnalystConfidenceRate.append(alystConfidenceRate)
    
    analystRatesList = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="profile-stat"]')))

    AnalystErrorRate.append(analystRatesList[0].text)
    AnalystAccPercentile.append(analystRatesList[1].text)
    AnalystPoints.append(analystRatesList[2].text)
    AnalystPtsPerEstimate.append(analystRatesList[3].text)
    AnalystStocks.append(analystRatesList[4].text)
    AnalystPendingEst.append(analystRatesList[5].text)
    
    get_Stocks_Covered(analystUserName)
    get_Pending_Estimates(analystUserName)
    get_Scored_Estimates(analystUserName)

In [10]:
def get_Stocks_Covered(analystUserName):
    element = browser.find_element(By.XPATH, '//div[@id="profile-covered-stocks-wrap"]//h2[@class="profile-secondary-header"]')
    actions = ActionChains(browser)
    actions.move_to_element(element).perform()

    isStocksCoveredExists = True
    scTable = browser.find_elements(By.XPATH, '//div[@id="profile-covered-stocks-wrap"]//div[@id="profile-covered-stocks"]')
    if len(scTable) == 0:
        isStocksCoveredExists = False
    
    if isStocksCoveredExists:
        scClick = browser.find_elements(By.XPATH, '//div[@id="profile-covered-stocks"]//a[@class="lnkd-tbl-show-more"]')
        if len(scClick) > 0:
            for i in scClick:
                i.click()

        scTickerList = browser.find_elements(By.XPATH, '//div[@class="td profile-stcks-tbl-instrument"]')
        for i in scTickerList:
            scAnalystName.append(analystUserName)
            scTicker.append(i.text)

        scReportsList = browser.find_elements(By.XPATH, '//div[contains(@class,"linked-row")]//div[@class="td profile-tbl-bordered"]')
        for i in scReportsList:
            scReports.append(i.text)

        scQuarterList = browser.find_elements(By.XPATH, '//div[contains(@class,"linked-row")]//div[@class="td profile-stcks-tbl-quarters"]')
        for i in scQuarterList:
            scQuarter.append(i.text)

        scPointsList = browser.find_elements(By.XPATH, '//div[contains(@class,"linked-row")]//div[@class="td profile-stcks-tbl-points"]')
        for i in scPointsList:
            scPoints.append(i.text)

        scPtsEtsList = browser.find_elements(By.XPATH, '//div[contains(@class,"linked-row")]//div[@class="td profile-stcks-tbl-points-per-estimate"]')
        for i in scPtsEtsList:
            scPtsEts.append(i.text)

        scErrorRateList = browser.find_elements(By.XPATH, '//div[contains(@class,"linked-row")]//div[@class="td profile-stcks-tbl-error-rate"]')
        for i in scErrorRateList:
            scErrorRate.append(i.text)

        scAccuracyList = browser.find_elements(By.XPATH, '//div[contains(@class,"linked-row")]//div[@class="td profile-stcks-tbl-accuracy"]')
        for i in scAccuracyList:
            scAccuracy.append(i.text)

In [11]:
def get_Pending_Estimates(analystUserName):    
    isPEsectionExists = True
    peSection = browser.find_elements(By.XPATH, '//div[@id="equities"]//div[@class="profile-left-section"][3]//p[@class="profile-section-message"]')
    if len(peSection) > 0:
        isPEsectionExists = False

    peClick = browser.find_elements(By.XPATH, '//div[@id="profile-pending-estimates"]//a[@class="lnkd-tbl-show-more"]')
    if len(peClick) > 0:
        for i in peClick:
            i.click()
    
    if isPEsectionExists:
        peTickerList = browser.find_elements(By.XPATH, '//div[@id="equities"]//div[@class="profile-left-section"][3]//a[contains(@class,"linked-row")]//strong[@class="profile-tbl-ticker"]')
        for i in peTickerList:
            peAnalystName.append(analystUserName)
            peTicker.append(i.text)

        peReportsList = browser.find_elements(By.XPATH, '//div[@id="equities"]//div[@class="profile-left-section"][3]//a[contains(@class,"linked-row")]//div[@class="td profile-tbl-bordered"]')
        for i in peReportsList:
            peReports.append(i.text)

        peQuarterList = browser.find_elements(By.XPATH, '//div[@id="equities"]//div[@class="profile-left-section"][3]//a[contains(@class,"linked-row")]//div[@class="td"][2]')
        for i in peQuarterList:
            peQuarter.append(i.text)

        pePublishedList = browser.find_elements(By.XPATH, '//div[@id="equities"]//div[@class="profile-left-section"][3]//a[contains(@class,"linked-row")]//div[@class="td"][3]')
        for i in pePublishedList:
            pePublished.append(i.text)

        peEPSList = browser.find_elements(By.XPATH, '//div[@id="equities"]//div[@class="profile-left-section"][3]//a[contains(@class,"linked-row")]//div[@class="td"][4]')
        for i in peEPSList:
            peEPS.append(i.text)

        peRevenueList = browser.find_elements(By.XPATH, '//div[@id="equities"]//div[@class="profile-left-section"][3]//a[contains(@class,"linked-row")]//div[@class="td"][5]')
        for i in peRevenueList:
            peRevenue.append(i.text)

In [12]:
def get_Scored_Estimates(analystUserName):
    isScoredEstExists = True
    seTable = browser.find_elements(By.XPATH, '//div[@id="profile-scored-estimates"]')
    if len(seTable) == 0:
        isScoredEstExists = False
    
    if isScoredEstExists:
        seClick = browser.find_elements(By.XPATH, '//div[@id="profile-scored-estimates"]//a[@class="lnkd-tbl-show-more"]')
        if len(seClick) > 0:
            for i in seClick:
                i.click()

        seTickerList = browser.find_elements(By.XPATH, '//div[@id="equities"]//div[@class="profile-left-section"][4]//a[contains(@class,"linked-row")]//strong[@class="profile-tbl-ticker"]')
        for i in seTickerList:
            seAnalystName.append(analystUserName)
            seTicker.append(i.text)

        seReportedList = browser.find_elements(By.XPATH, '//div[@id="equities"]//div[@class="profile-left-section"][4]//a[contains(@class,"linked-row")]//div[@class="td profile-tbl-bordered"]')
        for i in seReportedList:
            seReported.append(i.text)

        seQuarterList = browser.find_elements(By.XPATH, '//div[@id="equities"]//div[@class="profile-left-section"][4]//a[contains(@class,"linked-row")]//div[@class="td"][2]')
        for i in seQuarterList:
            seQuarter.append(i.text)

        seRankList = browser.find_elements(By.XPATH, '//div[@id="equities"]//div[@class="profile-left-section"][4]//a[contains(@class,"linked-row")]//div[@class="td"][3]')
        for i in seRankList:
            seRank.append(i.text)

        seEPSList = browser.find_elements(By.XPATH, '//div[@id="equities"]//div[@class="profile-left-section"][4]//a[contains(@class,"linked-row")]//div[@class="td"][4]')
        for i in seEPSList:
            seEPS.append(i.text)

        seRevenueList = browser.find_elements(By.XPATH, '//div[@id="equities"]//div[@class="profile-left-section"][4]//a[contains(@class,"linked-row")]//div[@class="td"][5]')
        for i in seRevenueList:
            seRevenue.append(i.text)

        seTotPointsList = browser.find_elements(By.XPATH, '//div[@id="equities"]//div[@class="profile-left-section"][4]//a[contains(@class,"linked-row")]//div[@class="td"][6]')
        for i in seTotPointsList:
            seTotPoints.append(i.text)

In [13]:
tickerListDF = pd.read_excel("TickerList.xlsx")

Ticker_BI, Ticker_Name, Ticker_Sectors, Ticker_Industries, Ticker_numFollowers, Ticker_NumAnalysts = ([] for i in range(6))
Ticker_EPS, Quarter_EPS, ReportedEarnings_EPS, EstConsensus_EPS, EstMean_EPS, WSConsensus_EPS = ([] for i in range(6))
Ticker_AE, Quarter_AE, AnalystDisplayName_AE, AnalystSecondaryName_AE, AnalystEPSestimation_AE = ([] for i in range(5))

AnalystUserName, AnalystRole, AnalystJoinDate, AnalystConfidenceRate, AnalystErrorRate, AnalystAccPercentile, AnalystPoints, AnalystPtsPerEstimate, AnalystStocks, AnalystPendingEst = ([] for i in range(10))
scAnalystName, scTicker, scReports, scQuarter, scPoints, scPtsEts, scErrorRate, scAccuracy  = ([] for i in range(8))
peAnalystName, peTicker, peQuarter, peReports, pePublished, peEPS, peRevenue  = ([] for i in range(7))
seAnalystName, seTicker, seQuarter, seReported, seRank, seEPS, seRevenue, seTotPoints  = ([] for i in range(8))

In [14]:
#from datetime import datetime
#print("start time =", datetime.now().time())

getCompanyInformation(tickerListDF)

uniqueAnalystList = list(set(AnalystSecondaryName_AE))
print(len(uniqueAnalystList))
for i in uniqueAnalystList:
    anlst_URL = ("https://www.estimize.com/users/"+i.lower())
    browser.get(anlst_URL)
    activeUser = browser.find_elements(By.XPATH, '//h1[@class="profile-primary-header"]')
    if len(activeUser) > 0:
        get_Analyst_Information()
#print("end time =", datetime.now().time())

start time = 22:49:45.145853
1651
done with analyst info
end time = 00:16:05.802484


In [15]:
companiesInfo = zip(Ticker_BI, Ticker_Name, Ticker_Sectors, Ticker_Industries, Ticker_numFollowers, Ticker_NumAnalysts)
EPSInfo = zip(Ticker_EPS, Quarter_EPS, ReportedEarnings_EPS, EstConsensus_EPS, EstMean_EPS, WSConsensus_EPS)
EPSAnalystsEstimation = zip(Ticker_AE, Quarter_AE, AnalystDisplayName_AE, AnalystSecondaryName_AE, AnalystEPSestimation_AE)

AnalystInformation = zip(AnalystUserName, AnalystRole, AnalystJoinDate, AnalystConfidenceRate, AnalystErrorRate, AnalystAccPercentile, AnalystPoints, AnalystPtsPerEstimate, AnalystStocks, AnalystPendingEst)
StocksCovered = zip(scAnalystName, scTicker, scReports, scQuarter, scPoints, scPtsEts, scErrorRate, scAccuracy)
PendingEstimation = zip(peAnalystName, peTicker, peQuarter, peReports, pePublished, peEPS, peRevenue)
ScoredEstimation = zip(seAnalystName, seTicker, seQuarter, seReported, seRank, seEPS, seRevenue, seTotPoints)

In [16]:
wb = openpyxl.Workbook()
wb['Sheet'].title = "CompaniesInfo"
sh1 = wb.active
sh1.append(["Ticker_BI", "Ticker_Name", "Ticker_Sectors", "Ticker_Industries", "Ticker_numFollowers", "Ticker_NumAnalysts"])
for info in list(companiesInfo):
    sh1.append(info)
    
ws1 = wb.create_sheet("Sheet1")
ws1.title = "EPSInfo"
ws1.append(["Ticker_EPS", "Quarter_EPS", "ReportedEarnings_EPS", "EstConsensus_EPS", "EstMean_EPS", "WSConsensus_EPS"])
for espInf in list(EPSInfo):
    ws1.append(espInf)
    
ws2 = wb.create_sheet("Sheet2")
ws2.title = "EPSAnalystsEstimation"
ws2.append(["Ticker_AE", "Quarter_AE", "AnalystDisplayName_AE", "AnalystSecondaryName_AE", "AnalystEPSestimation_AE"])
for espAE in list(EPSAnalystsEstimation):
    ws2.append(espAE)

ws3 = wb.create_sheet("Sheet3")
ws3.title = "AnalystInformation"
ws3.append(["AnalystUserName", "AnalystRole", "AnalystJoinDate", "AnalystConfidenceRate", "AnalystErrorRate",
            "AnalystAccPercentile", "AnalystPoints", "AnalystPtsPerEstimate", "AnalystStocks", "AnalystPendingEst"])
for anlytInf in list(AnalystInformation):
    ws3.append(anlytInf)
    
ws4 = wb.create_sheet("Sheet4")
ws4.title = "StocksCovered"
ws4.append(["scAnalystName", "scTicker", "scReports", "scQuarter", "scPoints", "scPtsEts", "scErrorRate", "scAccuracy"])
for stkCover in list(StocksCovered):
    ws4.append(stkCover)
    
ws5 = wb.create_sheet("Sheet5")
ws5.title = "PendingEstimation"
ws5.append(["peAnalystName", "peTicker", "peQuarter", "peReports", "pePublished", "peEPS", "peRevenue"])
for pndgEst in list(PendingEstimation):
    ws5.append(pndgEst)
    
ws6 = wb.create_sheet("Sheet6")
ws6.title = "ScoredEstimation"
ws6.append(["seAnalystName", "seTicker", "seQuarter", "seReported", "seRank", "seEPS", "seRevenue", "seTotPoints"])
for scrdEst in list(ScoredEstimation):
    ws6.append(scrdEst)
    
wb.save("MSA8040_Estimize.xlsx")
print("MSA8040_Estimize saved")    

MSA8040_Estimize saved


In [17]:
browser.quit()